# Project Summary: Black-Scholes Option Pricing and Implied Volatility

## 📈  Goal
Explore the Black-Scholes pricing model for European call options, compute option prices, invert the model to find implied volatility, and visualize the implied volatility surface using real market data.

---

## 🧮 A – Analytical Call Price

- Implement a Python function `Call(P0, r, sigma, T, K)` to compute the Black-Scholes price of a European call option.
- Use the parameters:  
  `P0 = 1`, `r = 0.05`, `sigma = 0.2`, `T = 2`, `K = 0.8`.

✅ **Output**: Analytical call price using closed-form formula.

---

## 🔁 B – Implied Volatility (Inversion)

- Implement a Python function `ImpliedVola(P0, r, T, K, C)` using `scipy.optimize.root`.
- The function should solve for `sigma` such that the call price matches a given market price `C`.
- Check correctness using the price from part (a).

✅ **Output**: Implied volatility that fits the call price.

---

## 📊 C – Real Market Data (EuroStoxx 50)

- Load `OptionValue.csv` containing call option prices for various strikes and maturities.
- Extract maturities, risk-free rates, and option prices.
- Use `P0 = 2890.62` (current EuroStoxx 50 price).
- Compute implied volatilities for all entries using the function from part (b).

✅ **Output**: Matrix of implied volatilities.

---

## 🌐 D – 3D Volatility Surface Plot

- Plot the implied volatility surface using `matplotlib`’s 3D plotting tools:
  ```python
  ax = fig.add_subplot(111, projection='3d')


In [31]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import root
import matplotlib.pyplot as plt

In [46]:
# A - Black Scholes call price
# parameters of BS pricing for a european call option
p0 = 1      # initial price
r = 0.05    # risk neutral drift
sigma = 0.2 # volatility
T = 2       # time to maturity
K = 0.8     # strike price

def BScall(p0, r, sigma,T,K):
    d1 = (np.log(p0 / K) + (r + 0.5 * sigma * sigma) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = p0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call

call = BScall(p0,r,sigma,T,K)

print(f'The risk-neutral price of a European call option in the Black-Scholes model with parameters:\n'
      f'initial price (P0) = {p0}\n'
      f'risk-neutral drift (r) = {r}\n'
      f'volatility (sigma) = {sigma}\n'
      f'time to maturity (T) = {T}\n'
      f'strike price (K) = {K}\n'
      f'is:\n'
      f'Call Price = {call:.4f}')

The risk-neutral price of a European call option in the Black-Scholes model with parameters:
initial price (P0) = 1
risk-neutral drift (r) = 0.05
volatility (sigma) = 0.2
time to maturity (T) = 2
strike price (K) = 0.8
is:
Call Price = 0.2912


In [ ]:
# B implied volatility
def ImpliedVola(p0,r,T,K,C):
    f = lambda x: (BScall(p0,r,x,T,K) - C)
    return root(fun=f, x0 = 1).x[0]     #taking sigma0=1.0 is a reasonable first guess
isigma = ImpliedVola(p0,r,T,K,call)
print(f'''Implied Volatility Calculation:
  Given call price: {call:.4f}
  Implied Volatility: {isigma:.4f}
  True Volatility: {sigma:.4f}''')


Implied Volatility Calculation:
  Given call price: 0.2912
  Implied Volatility: 0.2000
  True Volatility: 0.2000


In [48]:
# C computing implied volatilities 'OptionValue.csv'
optiondata = np.load('./OptionValue.csv')


FileNotFoundError: [Errno 2] No such file or directory: './OptionValue.csv'